In [24]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
## Codigo Princial
import pandas as pd
from pathlib import Path
import logging
from datetime import datetime

# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# --------- CONFIGURAÇÕES ---------
arquivo = Path(r"C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\1 -Comunicação de Dados TI 2025.xlsx")

centros_alvo = {
    "24573444", "24573614", "20570004", "24573434", "24571234", "24571394",
    "24571874", "24573454", "21570004", "24571384", "24602594", "24570004"
}

meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
         'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

abas_ignorar = {"CC", "Resumo"}

def validar_arquivo(caminho):
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    if not caminho.suffix.lower() in ['.xlsx', '.xls']:
        raise ValueError("Arquivo precisa ser Excel (.xlsx ou .xls)")

def converter_para_float(valor_bruto):
    try:
        valor_limpo = str(valor_bruto).strip().replace("R$", "").replace(" ", "")
        if "," in valor_limpo and "." in valor_limpo:
            valor_limpo = valor_limpo.replace(".", "").replace(",", ".")
        elif "," in valor_limpo:
            valor_limpo = valor_limpo.replace(",", ".")
        return float(valor_limpo)
    except (ValueError, AttributeError):
        return 0.0

def validar_valor(valor):
    if valor > 1_000_000_000:
        logging.warning(f"Valor suspeito encontrado: R$ {valor:,.2f}")
        return 0.0
    return valor

def gerar_planilha_organizada():
    dfs_organizados = {}
    xls = pd.ExcelFile(arquivo)
    
    for aba in xls.sheet_names:
        if aba.strip() in abas_ignorar:
            continue
            
        df = xls.parse(aba, header=None, dtype=str).fillna("")
        
        cabecalho_idx = None
        col_centro_idx = None
        
        for i, row in df.iterrows():
            for j, cel in enumerate(row):
                if "centro de custos" in str(cel).lower():
                    cabecalho_idx = i
                    col_centro_idx = j
                    break
            if cabecalho_idx is not None:
                break
                
        if col_centro_idx is None:
            logging.warning(f"Coluna 'Centro de Custos' não encontrada na aba {aba}")
            continue
            
        dados_mensais = {}
        coluna_inicial_meses = 5
        
        for linha in range(cabecalho_idx + 1, df.shape[0]):
            codigo = str(df.iat[linha, col_centro_idx]).strip()
            if not codigo:
                break
            if codigo in centros_alvo:
                if codigo not in dados_mensais:
                    dados_mensais[codigo] = {mes: 0.0 for mes in meses}
                
                for idx, mes in enumerate(meses):
                    valor_idx = coluna_inicial_meses + idx
                    valor_bruto = df.iat[linha, valor_idx]
                    valor_num = converter_para_float(valor_bruto)
                    valor_num = validar_valor(valor_num)
                    dados_mensais[codigo][mes] = valor_num
        
        if dados_mensais:
            linhas = []
            total_geral = 0
            
            for cc, valores in dados_mensais.items():
                linha = {'Centro de Custo': cc}
                linha.update(valores)
                total = sum(valores.values())
                total_geral += total
                linha['Total'] = total
                linhas.append(linha)
            
            totais_mensais = {}
            for mes in meses:
                totais_mensais[mes] = sum(dados_mensais[cc][mes] for cc in dados_mensais)
            
            linha_total = {'Centro de Custo': 'TOTAL'}
            linha_total.update(totais_mensais)
            linha_total['Total'] = total_geral
            linhas.append(linha_total)
            
            df_aba = pd.DataFrame(linhas)
            dfs_organizados[aba] = df_aba

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    arquivo_saida = arquivo.parent / f'Manutecao_Aplicativos_{timestamp}.xlsx'
    
    try:
        with pd.ExcelWriter(arquivo_saida, engine='xlsxwriter') as writer:
            for nome_aba, df in dfs_organizados.items():
                df.to_excel(writer, sheet_name=nome_aba, index=False)
                
                workbook = writer.book
                worksheet = writer.sheets[nome_aba]
                
                formato_moeda = workbook.add_format({
                    'num_format': 'R$ #,##0.00',
                    'bold': False,
                    'align': 'right'
                })
                
                formato_total = workbook.add_format({
                    'num_format': 'R$ #,##0.00',
                    'bold': True,
                    'align': 'right',
                    'bottom': 1,
                    'top': 1
                })
                
                formato_cabecalho = workbook.add_format({
                    'bold': True,
                    'align': 'center',
                    'valign': 'vcenter',
                    'bottom': 2
                })
                
                formato_texto = workbook.add_format({
                    'align': 'left',
                    'bold': False
                })
                
                formato_texto_total = workbook.add_format({
                    'align': 'left',
                    'bold': True
                })
                
                # Aplica formatos em todas as células
                for col in range(1, len(df.columns)):
                    worksheet.set_column(col, col, 12, formato_moeda)
                    for row in range(1, len(df) + 1):
                        valor = float(df.iloc[row-1, col])
                        formato = formato_total if row == len(df) else formato_moeda
                        worksheet.write(row, col, valor, formato)
                
                # Formata cabeçalhos
                for col, header in enumerate(df.columns):
                    worksheet.write(0, col, header, formato_cabecalho)
                
                # Formata coluna de Centro de Custo
                worksheet.set_column('A:A', 15)
                for row in range(1, len(df) + 1):
                    formato = formato_texto_total if row == len(df) else formato_texto
                    worksheet.write(row, 0, df.iloc[row-1, 0], formato)
                
                # Ajusta larguras das colunas
                worksheet.set_column('B:M', 12)
                worksheet.set_column('N:N', 15)
        
        print(f"\nNova planilha gerada: {arquivo_saida}")
        
    except PermissionError:
        print("\nERRO: Não foi possível salvar o arquivo.")
        print("Verifique se:")
        print("1. O arquivo não está aberto em outro programa")
        print("2. Você tem permissão para escrever na pasta")
        print(f"3. O caminho está acessível: {arquivo_saida}")
        raise

# Execução
validar_arquivo(arquivo)
gerar_planilha_organizada()

2025-06-11 16:37:26,659 - WARNING - Coluna 'Centro de Custos' não encontrada na aba Validador



Nova planilha gerada: C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\Manutecao_Aplicativos_20250611_163727.xlsx


In [21]:
## Verificação de CC
def verificar_centros_por_aba():
    """Verifica quais centros de custo estão presentes em cada aba"""
    
    # Carregamento do arquivo
    xls = pd.ExcelFile(arquivo)
    
    print("\n=== RELATÓRIO DE CENTROS DE CUSTO POR ABA ===\n")
    
    for aba in xls.sheet_names:
        if aba.strip() in abas_ignorar:
            continue
            
        # Carrega a aba atual
        df = xls.parse(aba, header=None, dtype=str).fillna("")
        
        # Procura a coluna de centro de custos
        cabecalho_idx = None
        col_centro_idx = None
        
        for i, row in df.iterrows():
            for j, cel in enumerate(row):
                if "centro de custos" in str(cel).lower():
                    cabecalho_idx = i
                    col_centro_idx = j
                    break
            if cabecalho_idx is not None:
                break
                
        if col_centro_idx is None:
            print(f"\nAba: {aba}")
            print("⚠️ Coluna 'Centro de Custos' não encontrada")
            continue
            
        # Lista todos os centros de custo encontrados na aba
        centros_encontrados = set()
        for linha in range(cabecalho_idx + 1, df.shape[0]):
            codigo = str(df.iat[linha, col_centro_idx]).strip()
            if not codigo:
                continue
            if codigo in centros_alvo:
                centros_encontrados.add(codigo)
        
        # Exibe os resultados para esta aba
        print(f"\nAba: {aba}")
        if centros_encontrados:
            print("Centros de Custo encontrados:")
            for cc in sorted(centros_encontrados):
                print(f"  • {cc}")
        else:
            print("❌ Nenhum centro de custo de interesse encontrado")

# Executa a verificação
verificar_centros_por_aba()


=== RELATÓRIO DE CENTROS DE CUSTO POR ABA ===


Aba: Validador
⚠️ Coluna 'Centro de Custos' não encontrada

Aba: 101Domains
❌ Nenhum centro de custo de interesse encontrado

Aba: Fapesp_NIC
❌ Nenhum centro de custo de interesse encontrado

Aba: Locaweb
❌ Nenhum centro de custo de interesse encontrado

Aba: InternetFAR
Centros de Custo encontrados:
  • 24570004
  • 24573614
  • 24602594

Aba: InternetSOB
Centros de Custo encontrados:
  • 20570004

Aba: InternetFOR
Centros de Custo encontrados:
  • 21570004

Aba: InternetCRA
❌ Nenhum centro de custo de interesse encontrado

Aba: InternetRider
❌ Nenhum centro de custo de interesse encontrado

Aba: InternetGAL
❌ Nenhum centro de custo de interesse encontrado

Aba: SBC
Centros de Custo encontrados:
  • 20570004
  • 21570004
  • 24570004
  • 24573614
  • 24602594


In [ ]:
#import pandas as pd
from pathlib import Path
import logging
from datetime import datetime

# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# --------- CONFIGURAÇÕES ---------
arquivo = Path(r"C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\Cópia de 1 -Comunicação de Dados TI 2025.xlsx")

centros_alvo = {
    "24573444", "24573614", "20570004", "24573434", "24571234", "24571394",
    "24571874", "24573454", "21570004", "24571384", "24602594", "24570004"
}

meses = ['Jan', 'Fev', 'Mar', 'Abr']
abas_ignorar = {"CC", "Resumo"}

# Funções auxiliares
def validar_arquivo(caminho):
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    if not caminho.suffix.lower() in ['.xlsx', '.xls']:
        raise ValueError("Arquivo precisa ser Excel (.xlsx ou .xls)")

def converter_para_float(valor_bruto):
    try:
        valor_limpo = str(valor_bruto).strip().replace("R$", "").replace(" ", "")
        if "," in valor_limpo and "." in valor_limpo:
            valor_limpo = valor_limpo.replace(".", "").replace(",", ".")
        elif "," in valor_limpo:
            valor_limpo = valor_limpo.replace(",", ".")
        return float(valor_limpo)
    except (ValueError, AttributeError):
        return 0.0

def validar_valor(valor):
    if valor > 1_000_000_000:
        logging.warning(f"Valor suspeito encontrado: R$ {valor:,.2f}")
        return 0.0
    return valor

def debug_valor_especifico(cc_teste="24570004"):
    """Debug específico para um CC"""
    xls = pd.ExcelFile(arquivo)
    total_cc = 0.0
    
    print(f"\n=== DEBUG DETALHADO PARA CC: {cc_teste} ===")
    
    for aba in xls.sheet_names:
        if aba.strip() in abas_ignorar:
            continue
        
        print(f"\nABA: {aba}")
        print("-" * 30)
        
        df = xls.parse(aba, header=None, dtype=str).fillna("")
        
        # Localiza coluna de centro de custos
        cabecalho_idx = None
        col_centro_idx = None
        col_meses = None
        
        # Procura cabeçalho e coluna dos meses
        for i, row in df.iterrows():
            for j, cel in enumerate(row):
                if "centro de custos" in str(cel).lower():
                    cabecalho_idx = i
                    col_centro_idx = j
                    
                    # Tenta encontrar coluna dos meses
                    for k, mes_cel in enumerate(row[j:]):
                        if str(mes_cel).strip().lower() == "jan":
                            col_meses = j + k
                            print(f"Coluna dos meses começa em: {k}")
                            break
                    break
            if cabecalho_idx is not None:
                break
        
        if col_centro_idx is None:
            continue
        
        # Procura valores do CC específico
        for linha in range(cabecalho_idx + 1, df.shape[0]):
            codigo = str(df.iat[linha, col_centro_idx]).strip()
            if not codigo:
                continue
            
            if codigo == cc_teste:
                print(f"\nLinha {linha}:")
                total_linha = 0.0
                
                # Tenta diferentes posições para os meses
                for pos_inicial in [5, 6, col_meses]:
                    if pos_inicial is None:
                        continue
                        
                    print(f"\nTestando coluna inicial: {pos_inicial}")
                    for idx, mes in enumerate(meses):
                        try:
                            valor_idx = pos_inicial + idx
                            valor_bruto = df.iat[linha, valor_idx]
                            valor_num = converter_para_float(valor_bruto)
                            if valor_num > 0:
                                print(f"  {mes} (col {valor_idx}): R$ {valor_num:,.2f}")
                                total_linha += valor_num
                        except:
                            continue
                
                if total_linha > 0:
                    print(f"Total desta linha: R$ {total_linha:,.2f}")
                    total_cc += total_linha
    
    print("\n" + "=" * 50)
    print(f"TOTAL GERAL PARA CC {cc_teste}: R$ {total_cc:,.2f}")
    print("=" * 50)
    
    return total_cc

def gerar_planilha_jan_abr():
    # ... código existente da função gerar_planilha_jan_abr ...

# Execução com debug
print("\nIniciando debug específico...")
total_encontrado = debug_valor_especifico("24570004")
print(f"\nTotal encontrado: R$ {total_encontrado:,.2f}")
print(f"Total esperado:   R$ 5.264,62")
print(f"Diferença:        R$ {5264.62 - total_encontrado:,.2f}")

print("\nGerando planilha...")
validar_arquivo(arquivo)
gerar_planilha_jan_abr()

2025-05-19 14:29:11,395 - WARNING - Coluna 'Centro de Custos' não encontrada na aba Validador



Nova planilha gerada: C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\Totais_Jan_Abr_20250519_142911.xlsx
Total geral Jan-Abr: R$ 4,138.30


In [26]:
## Debug do Codigo
import pandas as pd
from pathlib import Path
import logging
from datetime import datetime

# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# --------- CONFIGURAÇÕES ---------
arquivo = Path(r"C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\4 -Garantia Estendida Equip Informática.xlsx")

centros_alvo = {
    "20750005", "24750005", "21750005", "40750005", "24752445"   
}

meses = ['Jan', 'Fev', 'Mar', 'Abr']
abas_ignorar = {"CC", "Resumo"}

def debug_valores_detalhado():
    """Função para debug detalhado dos valores"""
    xls = pd.ExcelFile(arquivo)
    print("\n=== INICIANDO DEBUG DETALHADO ===")
    
    totais_por_cc = {}
    soma_total = 0.0  # Novo: variável para soma total
    
    
    for aba in xls.sheet_names:
        if aba.strip() in abas_ignorar:
            continue
            
        print(f"\nAnalisando aba: {aba}")
        df = xls.parse(aba, header=None, dtype=str).fillna("")
        
        # Localiza coluna de centro de custos
        cabecalho_idx = None
        col_centro_idx = None
        
        for i, row in df.iterrows():
            for j, cel in enumerate(row):
                if "centro de custos" in str(cel).lower():
                    cabecalho_idx = i
                    col_centro_idx = j
                    print(f"Encontrado cabeçalho na linha {i}, coluna {j}")
                    break
            if cabecalho_idx is not None:
                break
                
        if col_centro_idx is None:
            print("❌ Coluna de Centro de Custos não encontrada")
            continue
            
        # Processa valores
        for linha in range(cabecalho_idx + 1, df.shape[0]):
            codigo = str(df.iat[linha, col_centro_idx]).strip()
            if not codigo:
                continue
                
            if codigo in centros_alvo:
                if codigo not in totais_por_cc:
                    totais_por_cc[codigo] = {mes: 0.0 for mes in meses}
                
                print(f"\nCC: {codigo}")
                for idx, mes in enumerate(meses):
                    valor_idx = 5 + idx  # coluna inicial + índice do mês
                    valor_bruto = df.iat[linha, valor_idx]
                    valor_num = converter_para_float(valor_bruto)
                    
                    if valor_num > 0:
                        print(f"  {mes}: R$ {valor_num:,.2f}")
                        totais_por_cc[codigo][mes] += valor_num

   # Mostra totais
    print("\n=== TOTAIS POR CENTRO DE CUSTO ===")
    for cc, valores in totais_por_cc.items():
        total_cc = sum(valores.values())
        soma_total += total_cc  # Novo: acumula o total
        print(f"\nCC: {cc}")
        for mes, valor in valores.items():
            if valor > 0:
                print(f"  {mes}: R$ {valor:,.2f}")
        print(f"  TOTAL: R$ {total_cc:,.2f}")
    
    # Novo: Mostra soma total
    print("\n" + "=" * 50)
    print(f"SOMA TOTAL DE TODOS OS CCs: R$ {soma_total:,.2f}")
    print("=" * 50)

# Execução
print("\nIniciando verificação...")
debug_valores_detalhado()

print("\nGerando planilha...")
validar_arquivo(arquivo)
gerar_planilha_jan_abr()


Iniciando verificação...

=== INICIANDO DEBUG DETALHADO ===

Analisando aba: Validador
❌ Coluna de Centro de Custos não encontrada

Analisando aba: IK1
Encontrado cabeçalho na linha 5, coluna 2

CC: 20750005
  Jan: R$ 10.56
  Fev: R$ 10.56
  Mar: R$ 10.56
  Abr: R$ 10.56

CC: 24750005
  Jan: R$ 112.06
  Fev: R$ 112.06
  Mar: R$ 112.06
  Abr: R$ 112.06

CC: 40750005
  Jan: R$ 10.56
  Fev: R$ 10.56
  Mar: R$ 10.56
  Abr: R$ 10.56

Analisando aba: Sercompe_1
Encontrado cabeçalho na linha 5, coluna 2

CC: 20750005
  Jan: R$ 489.03
  Fev: R$ 489.03
  Mar: R$ 489.03
  Abr: R$ 489.03

CC: 21750005
  Jan: R$ 29.64
  Fev: R$ 29.64
  Mar: R$ 29.64
  Abr: R$ 29.64

CC: 24750005
  Jan: R$ 681.68
  Fev: R$ 681.68
  Mar: R$ 681.68
  Abr: R$ 681.68

CC: 24752445
  Jan: R$ 111.14
  Fev: R$ 111.14
  Mar: R$ 111.14
  Abr: R$ 111.14

CC: 40750005
  Jan: R$ 51.87
  Fev: R$ 51.87
  Mar: R$ 51.87
  Abr: R$ 51.87

Analisando aba: Sercompe_2
Encontrado cabeçalho na linha 5, coluna 2

CC: 21750005
  Jan: R$ 1

2025-05-19 17:33:09,560 - WARNING - Coluna 'Centro de Custos' não encontrada na aba Validador


Encontrado cabeçalho na linha 5, coluna 2

CC: 20750005

=== TOTAIS POR CENTRO DE CUSTO ===

CC: 20750005
  Jan: R$ 1,087.70
  Fev: R$ 1,087.70
  Mar: R$ 1,087.70
  Abr: R$ 1,087.70
  TOTAL: R$ 4,350.82

CC: 24750005
  Jan: R$ 11,412.15
  Fev: R$ 11,412.15
  Mar: R$ 11,412.15
  Abr: R$ 11,412.15
  TOTAL: R$ 45,648.61

CC: 40750005
  Jan: R$ 1,520.27
  Fev: R$ 1,520.27
  Mar: R$ 1,520.27
  Abr: R$ 1,520.27
  TOTAL: R$ 6,081.08

CC: 21750005
  Jan: R$ 1,470.71
  Fev: R$ 1,470.71
  Mar: R$ 1,470.71
  Abr: R$ 1,470.71
  TOTAL: R$ 5,882.85

CC: 24752445
  Jan: R$ 160.55
  Fev: R$ 160.55
  Mar: R$ 160.55
  Abr: R$ 160.55
  TOTAL: R$ 642.20

SOMA TOTAL DE TODOS OS CCs: R$ 62,605.56

Gerando planilha...

Nova planilha gerada: C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\Totais_Jan_Abr_20250519_173309.xlsx
Total geral Jan-Abr: R$ 62,605.56


In [1]:
## Gerar o relatorio por contrato
import pandas as pd
from pathlib import Path
import logging
from datetime import datetime

# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# --------- CONFIGURAÇÕES ---------
arquivo = Path(r"C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\4 -Garantia Estendida Equip Informática.xlsx")

centros_alvo = {
    "20750005", "24750005", "21750005", "40750005", "24752445"}

# Lista completa dos meses
meses = ['Jan', 'Fev', 'Mar', 'Abr']

abas_ignorar = {"CC", "Resumo"}

def validar_arquivo(caminho):
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    if not caminho.suffix.lower() in ['.xlsx', '.xls']:
        raise ValueError("Arquivo precisa ser Excel (.xlsx ou .xls)")

def converter_para_float(valor_bruto):
    try:
        valor_limpo = str(valor_bruto).strip().replace("R$", "").replace(" ", "")
        if "," in valor_limpo and "." in valor_limpo:
            valor_limpo = valor_limpo.replace(".", "").replace(",", ".")
        elif "," in valor_limpo:
            valor_limpo = valor_limpo.replace(",", ".")
        return float(valor_limpo)
    except (ValueError, AttributeError):
        return 0.0

def gerar_resumo_contratos():
    """Gera resumo dos contratos com totais por CC"""
    xls = pd.ExcelFile(arquivo)
    resumo_contratos = []
    
    for aba in xls.sheet_names:
        if aba.strip() in abas_ignorar:
            continue
            
        print(f"\nProcessando aba: {aba}")
        df = xls.parse(aba, header=None, dtype=str).fillna("")
        
        # Localiza coluna de centro de custos
        cabecalho_idx = None
        col_centro_idx = None
        col_meses = None
        
        # Procura cabeçalho e coluna dos meses
        for i, row in df.iterrows():
            for j, cel in enumerate(row):
                if "centro de custos" in str(cel).lower():
                    cabecalho_idx = i
                    col_centro_idx = j
                    
                    # Procura coluna Janeiro
                    for k, mes_cel in enumerate(row[j:]):
                        if str(mes_cel).strip().lower() == "jan":
                            col_meses = j + k
                            print(f"Coluna dos meses começa em: {col_meses}")
                            break
                    break
            if cabecalho_idx is not None:
                break
        
        if col_centro_idx is None:
            print(f"⚠️ Aba {aba}: Coluna 'Centro de Custos' não encontrada")
            continue
            
        # Processa valores
        total_aba = 0.0
        ccs_encontrados = set()
        
        for linha in range(cabecalho_idx + 1, df.shape[0]):
            codigo = str(df.iat[linha, col_centro_idx]).strip()
            if not codigo:
                continue
                
            if codigo in centros_alvo:
                ccs_encontrados.add(codigo)
                total_linha = 0.0
                
                # Soma valores de todos os meses
                for idx, mes in enumerate(meses):
                    try:
                        valor_idx = (col_meses or 5) + idx
                        valor_bruto = df.iat[linha, valor_idx]
                        valor_num = converter_para_float(valor_bruto)
                        if valor_num > 0:
                            total_linha += valor_num
                            print(f"  {codigo} - {mes}: R$ {valor_num:,.2f}")
                    except:
                        continue
                
                total_aba += total_linha
                print(f"  Total {codigo}: R$ {total_linha:,.2f}")
        
        if ccs_encontrados:
            resumo_contratos.append({
                'Contrato': aba,
                'CCs Encontrados': ', '.join(sorted(ccs_encontrados)),
                'Total': total_aba
            })
            print(f"  ✓ Total da aba: R$ {total_aba:,.2f}")
    
    if not resumo_contratos:
        print("❌ Nenhum centro de custo encontrado em nenhuma aba")
        return
    
    # Cria DataFrame de resumo
    df_resumo = pd.DataFrame(resumo_contratos)
    
    # Calcula total geral
    total_geral = df_resumo['Total'].sum()
    
    # Adiciona linha de total
    df_resumo.loc[len(df_resumo)] = ['TOTAL GERAL', '', total_geral]
    
    # Gera arquivo Excel
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    arquivo_saida = arquivo.parent / f'Resumo_Contratos_Garantia_Estendida_Equip_Infomartica{timestamp}.xlsx'
    
    try:
        with pd.ExcelWriter(arquivo_saida, engine='xlsxwriter') as writer:
            df_resumo.to_excel(writer, sheet_name='Resumo', index=False)
            
            workbook = writer.book
            worksheet = writer.sheets['Resumo']
            
            # Formatos
            formato_moeda = workbook.add_format({
                'num_format': 'R$ #,##0.00',
                'align': 'right'
            })
            
            formato_total = workbook.add_format({
                'num_format': 'R$ #,##0.00',
                'bold': True,
                'align': 'right',
                'top': 1,
                'bottom': 1
            })
            
            formato_cabecalho = workbook.add_format({
                'bold': True,
                'align': 'center',
                'valign': 'vcenter',
                'bottom': 2
            })
            
            formato_texto = workbook.add_format({
                'align': 'left'
            })
            
            formato_texto_total = workbook.add_format({
                'align': 'left',
                'bold': True
            })
            
            # Aplica formatos
            worksheet.set_column('A:A', 40, formato_texto)  # Contrato
            worksheet.set_column('B:B', 50, formato_texto)  # CCs
            worksheet.set_column('C:C', 15, formato_moeda)  # Total
            
            # Formata cabeçalhos
            for col, header in enumerate(df_resumo.columns):
                worksheet.write(0, col, header, formato_cabecalho)
            
            # Formata linha de total
            ultima_linha = len(df_resumo)
            worksheet.write(ultima_linha, 0, 'TOTAL GERAL', formato_texto_total)
            worksheet.write(ultima_linha, 2, total_geral, formato_total)
        
        print(f"\n✅ Arquivo gerado: {arquivo_saida}")
        print(f"Total Geral: R$ {total_geral:,.2f}")
        
    except Exception as e:
        print(f"\n❌ Erro ao gerar arquivo: {e}")
        raise

# Execução
print("\nIniciando processamento...")
validar_arquivo(arquivo)
gerar_resumo_contratos()


Iniciando processamento...

Processando aba: Validador
⚠️ Aba Validador: Coluna 'Centro de Custos' não encontrada

Processando aba: IK1
  20750005 - Jan: R$ 10.56
  20750005 - Fev: R$ 10.56
  20750005 - Mar: R$ 10.56
  20750005 - Abr: R$ 10.56
  Total 20750005: R$ 42.24
  24750005 - Jan: R$ 112.06
  24750005 - Fev: R$ 112.06
  24750005 - Mar: R$ 112.06
  24750005 - Abr: R$ 112.06
  Total 24750005: R$ 448.24
  40750005 - Jan: R$ 10.56
  40750005 - Fev: R$ 10.56
  40750005 - Mar: R$ 10.56
  40750005 - Abr: R$ 10.56
  Total 40750005: R$ 42.24
  ✓ Total da aba: R$ 532.72

Processando aba: Sercompe_1
  20750005 - Jan: R$ 489.03
  20750005 - Fev: R$ 489.03
  20750005 - Mar: R$ 489.03
  20750005 - Abr: R$ 489.03
  Total 20750005: R$ 1,956.13
  21750005 - Jan: R$ 29.64
  21750005 - Fev: R$ 29.64
  21750005 - Mar: R$ 29.64
  21750005 - Abr: R$ 29.64
  Total 21750005: R$ 118.55
  24750005 - Jan: R$ 681.68
  24750005 - Fev: R$ 681.68
  24750005 - Mar: R$ 681.68
  24750005 - Abr: R$ 681.68
  Tota

In [ ]:
## Gerar o relatorio por contrato por meses especificos
import pandas as pd
from pathlib import Path
import logging
from datetime import datetime

# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# --------- CONFIGURAÇÕES ---------
arquivo = Path(r"C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\4 -Garantia Estendida Equip Informática.xlsx")

centros_alvo = {
    "20750005", "24750005", "21750005", "40750005", "24752445"}

# Lista completa dos meses
meses = ['Jan', 'Fev', 'Mar', 'Abr']

abas_ignorar = {"CC", "Resumo"}

def validar_arquivo(caminho):
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    if not caminho.suffix.lower() in ['.xlsx', '.xls']:
        raise ValueError("Arquivo precisa ser Excel (.xlsx ou .xls)")

def converter_para_float(valor_bruto):
    try:
        valor_limpo = str(valor_bruto).strip().replace("R$", "").replace(" ", "")
        if "," in valor_limpo and "." in valor_limpo:
            valor_limpo = valor_limpo.replace(".", "").replace(",", ".")
        elif "," in valor_limpo:
            valor_limpo = valor_limpo.replace(",", ".")
        return float(valor_limpo)
    except (ValueError, AttributeError):
        return 0.0

def gerar_resumo_contratos():
    """Gera resumo dos contratos com totais mensais"""
    xls = pd.ExcelFile(arquivo)                                                                      
    resumo_contratos = []
    
    for aba in xls.sheet_names:
        if aba.strip() in abas_ignorar:
            continue
            
        print(f"\nProcessando aba: {aba}")
        df = xls.parse(aba, header=None, dtype=str).fillna("")
        
        # Localiza coluna de centro de custos
        cabecalho_idx = None
        col_centro_idx = None
        col_meses = None
        
        # Procura cabeçalho e coluna dos meses
        for i, row in df.iterrows():
            for j, cel in enumerate(row): 
                if "centro de custos" in str(cel).lower():
                    cabecalho_idx = i
                    col_centro_idx = j
                    
                    # Procura coluna Janeiro
                    for k, mes_cel in enumerate(row[j:]):
                        if str(mes_cel).strip().lower() == "jan":
                            col_meses = j + k
                            print(f"Coluna dos meses começa em: {col_meses}")
                            break
                    break
            if cabecalho_idx is not None:
                break
        
        if col_centro_idx is None:
            print(f"⚠️ Aba {aba}: Coluna 'Centro de Custos' não encontrada")
            continue
            
        # Dicionário para armazenar valores mensais
        valores_mensais = {mes: 0.0 for mes in meses}
        ccs_encontrados = set()
        
        for linha in range(cabecalho_idx + 1, df.shape[0]):
            codigo = str(df.iat[linha, col_centro_idx]).strip()
            if not codigo:
                continue
                
            if codigo in centros_alvo:
                ccs_encontrados.add(codigo)
                
                # Soma valores por mês
                for idx, mes in enumerate(meses):
                    try:
                        valor_idx = (col_meses or 5) + idx
                        valor_bruto = df.iat[linha, valor_idx]
                        valor_num = converter_para_float(valor_bruto)
                        if valor_num > 0:
                            valores_mensais[mes] += valor_num
                            print(f"  {codigo} - {mes}: R$ {valor_num:,.2f}")
                    except:
                        continue
        
        if ccs_encontrados:
            total_aba = sum(valores_mensais.values())
            dados_contrato = {
                'Contrato': aba,
                'CCs Encontrados': ', '.join(sorted(ccs_encontrados)),
            }
            # Adiciona valores mensais
            for mes in meses:
                dados_contrato[mes] = valores_mensais[mes]
            dados_contrato['Total'] = total_aba
            
            resumo_contratos.append(dados_contrato)
            print(f"\n  ✓ Valores mensais para {aba}:")
            for mes, valor in valores_mensais.items():
                print(f"    {mes}: R$ {valor:,.2f}")
            print(f"    Total: R$ {total_aba:,.2f}")
    
    if not resumo_contratos:
        print("❌ Nenhum centro de custo encontrado em nenhuma aba")
        return
    
    # Cria DataFrame de resumo
    df_resumo = pd.DataFrame(resumo_contratos)
    
    # Calcula totais mensais
    totais_mensais = {mes: df_resumo[mes].sum() for mes in meses}
    total_geral = df_resumo['Total'].sum()
    
    # Adiciona linha de total
    dados_total = {'Contrato': 'TOTAL GERAL', 'CCs Encontrados': ''}
    dados_total.update(totais_mensais)
    dados_total['Total'] = total_geral
    df_resumo.loc[len(df_resumo)] = dados_total
    
    # Gera arquivo Excel
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    arquivo_saida = arquivo.parent / f'Resumo_Contratos_Mensais_{timestamp}.xlsx'
    
    try:
        with pd.ExcelWriter(arquivo_saida, engine='xlsxwriter') as writer:
            df_resumo.to_excel(writer, sheet_name='Resumo', index=False)
            
            workbook = writer.book
            worksheet = writer.sheets['Resumo']
            
            # Formatos
            formato_moeda = workbook.add_format({
                'num_format': 'R$ #,##0.00',
                'align': 'right'
            })
            
            formato_total = workbook.add_format({
                'num_format': 'R$ #,##0.00',
                'bold': True,
                'align': 'right',
                'top': 1,
                'bottom': 1
            })
            
            formato_cabecalho = workbook.add_format({
                'bold': True,
                'align': 'center',
                'valign': 'vcenter',
                'bottom': 2
            })
            
            formato_texto = workbook.add_format({
                'align': 'left'
            })
            
            formato_texto_total = workbook.add_format({
                'align': 'left',
                'bold': True
            })
            
            # Aplica formatos
            worksheet.set_column('A:A', 40, formato_texto)  # Contrato
            worksheet.set_column('B:B', 50, formato_texto)  # CCs
            worksheet.set_column('C:F', 15, formato_moeda)  # Meses
            worksheet.set_column('G:G', 15, formato_moeda)  # Total
            
            # Formata cabeçalhos
            for col, header in enumerate(df_resumo.columns):
                worksheet.write(0, col, header, formato_cabecalho)
            
            # Formata linha de total
            ultima_linha = len(df_resumo)
            worksheet.write(ultima_linha, 0, 'TOTAL GERAL', formato_texto_total)
            for col in range(2, len(df_resumo.columns)):
                worksheet.write(ultima_linha, col, df_resumo.iloc[-1, col], formato_total)
        
        print(f"\n✅ Arquivo gerado: {arquivo_saida}")
        print(f"Total Geral: R$ {total_geral:,.2f}")
        
    except Exception as e:
        print(f"\n❌ Erro ao gerar arquivo: {e}")
        raise

# Execução
print("\nIniciando processamento...")
validar_arquivo(arquivo)
gerar_resumo_contratos()


Iniciando processamento...

Processando aba: Validador
⚠️ Aba Validador: Coluna 'Centro de Custos' não encontrada

Processando aba: IK1
  20750005 - Jan: R$ 10.56
  20750005 - Fev: R$ 10.56
  20750005 - Mar: R$ 10.56
  20750005 - Abr: R$ 10.56
  24750005 - Jan: R$ 112.06
  24750005 - Fev: R$ 112.06
  24750005 - Mar: R$ 112.06
  24750005 - Abr: R$ 112.06
  40750005 - Jan: R$ 10.56
  40750005 - Fev: R$ 10.56
  40750005 - Mar: R$ 10.56
  40750005 - Abr: R$ 10.56

  ✓ Valores mensais para IK1:
    Jan: R$ 133.18
    Fev: R$ 133.18
    Mar: R$ 133.18
    Abr: R$ 133.18
    Total: R$ 532.72

Processando aba: Sercompe_1
  20750005 - Jan: R$ 489.03
  20750005 - Fev: R$ 489.03
  20750005 - Mar: R$ 489.03
  20750005 - Abr: R$ 489.03
  21750005 - Jan: R$ 29.64
  21750005 - Fev: R$ 29.64
  21750005 - Mar: R$ 29.64
  21750005 - Abr: R$ 29.64
  24750005 - Jan: R$ 681.68
  24750005 - Fev: R$ 681.68
  24750005 - Mar: R$ 681.68
  24750005 - Abr: R$ 681.68
  24752445 - Jan: R$ 111.14
  24752445 - Fev: 